In [0]:
# Colab library to upload files to notebook
from google.colab import files

# Install Kaggle library
!pip install -q kaggle

In [0]:
!ls -lha kaggle.json

-rw-r--r-- 1 root root 64 May 10 18:44 kaggle.json


In [0]:
!pip install -q kaggle

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [0]:
!chmod 600 ~/.kaggle/kaggle.json

In [0]:
!kaggle datasets list

In [5]:
!kaggle  datasets download -d laurentmih/aisegmentcom-matting-human-datasets

100% 28.8G/28.8G [12:19<00:00, 31.9MB/s]
100% 28.8G/28.8G [12:19<00:00, 41.8MB/s]


In [0]:
import tensorflow as tf
import keras
from tensorflow.keras.callbacks import TensorBoard
import time
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd
from imutils import paths
import os
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths

# !unzip  '/content/aisegmentcom-matting-human-datasets.zip'

In [0]:
directory = '/content/clip_img'
new_directory = '/content/Images'


def copy_images():
    for file_name in os.listdir(directory):
        sub_dir_path = directory + '/' + file_name
        if (os.path.isdir(sub_dir_path)):
          for file_name1 in os.listdir(sub_dir_path):
              sub_dir_path1 = sub_dir_path + '/' + file_name1            
              if (os.path.isdir(sub_dir_path1)):
                     for image_name in os.listdir(sub_dir_path1):
                         if image_name[-4:] == '.jpg':
                            img = cv2.imread(sub_dir_path1+'/'+image_name)                            
                            copied_image_path = new_directory + '/' + 'person_'+image_name
                            # print(copied_image_path)
                            if img.size !=0:
                                cv2.imwrite(copied_image_path, img)

copy_images()


In [0]:
# for image_name in os.listdir('/content/Images'):
                        
#                             img = '/content/Images'+'/'+image_name
#                             # print(img)                           
#                             os.remove(img)

In [34]:
imagePaths = list(paths.list_images('/content/Images'))
data = []
labels = []
 
for imagePath in imagePaths:
	# extract the class label from the filename
	label = imagePath.split(os.path.sep)[-1]
	image = load_img(imagePath, target_size=(224, 224),grayscale=True,color_mode='gray')
	image = img_to_array(image)
	image = preprocess_input(image)

	# update the data and labels lists, respectively
	data.append(image)
	labels.append('0')
 
# convert the data and labels to NumPy arrays
data = np.array(data, dtype="float32") #ndarray: data ndarray with shape (3548, 224, 224, 3)


labels = np.array(labels)   #ndarray: labels ndarray with shape (1, 3548)
# labels =(np.array(labels).reshape(1,-1))
# print(label)

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/utils.py:104: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


In [0]:
labels =(np.array(labels).reshape(1,-1))

In [0]:
 
plt.imshow(data[1])
print(len(labels))
print(labels[2])


In [0]:
# data = []
# labels = []

In [0]:
 
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

In [0]:
nsamples, nx, ny = data.shape
d2_train_dataset = data.reshape((nsamples,nx*ny))

In [0]:

 
(x_train, x_test, y_train, y_test) = train_test_split(data, labels,test_size=0.20, random_state=42)

In [36]:
print(len(y_train))
print(len(x_train))
print(x_train.shape)
print(y_train.shape)
# plt.imshow(x_train[1])

2838
2838
(2838, 224, 224, 1)
(2838,)


In [37]:
 
x_train = tf.keras.utils.normalize(x_train, axis = 1)
y_train = np.array(y_train)
# x_train = tf.keras.utils.normalize(x_train, axis = 1)
x_test = tf.keras.utils.normalize(x_test,axis = 1)
# x_train = tf.keras.utils.normalize(x_train, axis = 1)
# x_test = tf.keras.utils.normalize(x_test,axis = 1)
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten()) 
model.add(tf.keras.layers.Dense(754,activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(600,activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(64,activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(10,activation=tf.nn.softmax)) 
model.compile(optimizer = "adam" , loss = 'categorical_crossentropy' , metrics = ['accuracy'] )
model.fit(x_train,y_train,epochs = 20 )
predictions = model.predict(x_test)
val_loss, val_acc = model.evaluate(x_test, y_test)

Epoch 1/20


ValueError: ignored

In [0]:
y_train